In [1]:
import os
import numpy as np
import pandas as pd
import difflib as dl
import subprocess
import os

# make sure we are at the top of the repo
wd = subprocess.check_output('git rev-parse --show-toplevel', shell = True)
os.chdir(wd[:-1]) #-1 removes \n

In [3]:
# bring in and clean wapo data
wapo = pd.read_csv('Data/PostKillingsData.csv')
wapo['date'] = pd.to_datetime(wapo['date'])
wapo = wapo.loc[:,['name','date','age','gender','race','city','state']]
wapo = wapo.sort(['date','name'])
wapo = wapo[wapo['date'] < '2017-01-01']
wapo = wapo.reset_index()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [5]:
# bring in and clean fbi data
# https://mappingpoliceviolence.org/s/MPVDatasetDownload-d4a6.xlsx convert first tab to csv
fbi = pd.read_csv('Data/MPVDatasetDownload.csv')
fbi = fbi.iloc[:,[0,1,2,3,5,7,8]]
fbi.columns = ['name','age','gender','race','date','city','state']
fbi = fbi.loc[:,['name','date','age','gender','race','city','state']]
fbi['date'] = pd.to_datetime(fbi['date'])
fbi['age'][fbi['age'] == 'Unknown'] = 0
fbi['age'] = fbi['age'].astype(float)
fbi = fbi.sort(['date','name'])
fbi = fbi[fbi['date'] > '2015-01-01']
fbi = fbi[fbi['date'] < '2017-01-01']
fbi = fbi.reset_index()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [4]:
# how much can we match by name only?
print 'We can match {0:.0f}% based on name only'.format(100*len(wapo[np.in1d(wapo['name'], fbi['name'])]) / float(len(wapo)))
# not very much. spelling differences, middle names, etc

We can match 68% based on name only


In [5]:
# we need to make columns match better
print wapo['race'].unique()
print fbi['race'].unique()

['W' 'A' 'H' 'B' 'O' nan 'N']
['White' 'Asian' 'Hispanic' 'Black' 'Unknown race' 'Pacific Islander'
 'Native American']


In [6]:
# how to best match race?
print '{0:.0f}% of Wapo records are W/A/H/B'.format(100*len(wapo[np.in1d(wapo['race'], ['W','A','H','B'])])/float(len(wapo)))
print '{0:.0f}% of FBI records are W/A/H/B'.format(100*len(fbi[np.in1d(fbi['race'], 
                                                                        ['White','Asian','Hispanic','Black'])])/float(len(fbi)))

93% of Wapo records are W/A/H/B
93% of FBI records are W/A/H/B


In [7]:
# clean up race columns
race_dict = {'W':'White',
            'A':'Asian',
            'H':'Hispanic',
            'B':'Black',
            'O':'Other'}
wapo['race'][~np.in1d(wapo['race'],['W','A','H','B'])] = 'O'
fbi['race'][~np.in1d(fbi['race'],['White','Asian','Hispanic','Black'])] = 'Other'

wapo['race'] = [race_dict[i] for i in wapo['race']]

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
# how to best match gender?

print 'Wapo gender categories: {}'.format(wapo['gender'].unique())
print 'FBI gender categories: {}'.format(fbi['gender'].unique())

print '{0:.0f}% of Wapo records are M/F'.format(100*len(wapo[np.in1d(wapo['gender'], ['M','F'])])/float(len(wapo)))
print '{0:.0f}% of FBI records are M/F'.format(100*len(fbi[np.in1d(fbi['gender'], 
                                                                        ['Male','Female'])])/float(len(fbi)))

Wapo gender categories: ['M' 'F']
FBI gender categories: ['Male' 'Female' 'Unknown' 'Transgender' nan]
100% of Wapo records are M/F
100% of FBI records are M/F


In [9]:
# hmm what's going on?
fbi[~np.in1d(fbi['gender'], ['Male','Female'])]

,index,name,date,age,gender,race,city,state
46,2299,Name withheld by police,2015-01-16,0.0,Unknown,Other,Mabank,TX
1102,1243,Tuan Hoang,2015-11-30,0.0,Unknown,Asian,Aurora,CO
1316,1030,Kayden Clarke transitioning from Danielle Jacobs,2016-02-04,24.0,Transgender,White,Mesa,AZ
1802,542,Name withheld by police,2016-07-04,29.0,NaN,Black,Rosser,TX


In [10]:
wapo[np.in1d(wapo['name'], ['Tuan Hoang','Kayden Clarke','Danielle Jacobs'])]

,index,name,date,age,gender,race,city,state
901,899,Tuan Hoang,2015-11-30,25.0,M,Asian,Aurora,CO
1080,1078,Kayden Clarke,2016-02-04,24.0,M,White,Mesa,AZ


In [11]:
wapo[np.in1d(wapo['city'], ['Mabank','Rosser'])]

,index,name,date,age,gender,race,city,state
37,37,Scott Hall,2015-01-16,41.0,M,White,Mabank,TX
1499,1500,TK TK,2016-07-04,NaN,M,Other,Rosser,TX


In [12]:
# all non M/F FBI records are classified as M by wapo. For ease of matching in this analysis, I will take Wapo's answer
fbi['gender'][~np.in1d(fbi['gender'],['Male','Female'])] = 'Male'

# and let's make them match
gender_dict = {'M':'Male',
               'F':'Female'}
wapo['gender'] = [gender_dict[i] for i in wapo['gender']]

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [13]:
# all columns should match now. 
wapo.head()

,index,name,date,age,gender,race,city,state
0,1,Lewis Lee Lembke,2015-01-02,47.0,Male,White,Aloha,OR
1,0,Tim Elliot,2015-01-02,53.0,Male,Asian,Shelton,WA
2,2,John Paul Quintero,2015-01-03,23.0,Male,Hispanic,Wichita,KS
3,5,Kenneth Joe Brown,2015-01-04,18.0,Male,White,Guthrie,OK
4,3,Matthew Hoffman,2015-01-04,32.0,Male,White,San Francisco,CA


In [14]:
# string concat to create id variable
wapo['id'] = wapo.apply(lambda x:'%s%s%s%s%s%s%s' % (x['name'],x['date'],x['age'],x['gender'],x['race'],x['city'],x['state']),axis=1)
fbi['id'] = fbi.apply(lambda x:'%s%s%s%s%s%s%s' % (x['name'],x['date'],x['age'],x['gender'],x['race'],x['city'],x['state']),axis=1)

In [15]:
# how many perfect matches
matched = [np.in1d(i, fbi['id']) for i in wapo['id']]

In [16]:
print 'We can match {0:.0f}% based on perfect id'.format(100*len(wapo[np.in1d(wapo['id'], fbi['id'])]) / float(len(wapo)))

We can match 45% based on perfect id


In [18]:
# of course this is worse than name only. but let's try some fuzzy matching

def best_match(x, y):
    rats = [dl.SequenceMatcher(None, x, i).ratio() for i in y]
    return([max(rats), y[rats.index(max(rats))]])

In [19]:
# what does FBI report that WAPO missed?
# slow... half an hour maybe?
fbi[['score','wapo_id']] = fbi['id'].apply(lambda x: best_match(x, wapo['id'])).apply(pd.Series)

In [31]:
fbi.sort('score').iloc[450:455,:]

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,index,name,date,age,gender,race,city,state,id,score,wapo_id
1539,805,Luis Soto,2016-04-10,29.0,Male,Hispanic,Memphis,TN,Luis Soto2016-04-10 00:00:0029.0MaleHispanicMe...,0.803571,Ricardo Tenorio2016-09-09 00:00:0032.0MaleHisp...
2194,153,William Ray Score Jr.,2016-11-11,47.0,Male,White,NaN,CO,William Ray Score Jr.2016-11-11 00:00:0047.0Ma...,0.806452,William Ray Score2016-11-11 00:00:0047.0MaleOt...
244,2100,Justin Fowler,2015-03-18,24.0,Male,Other,Lukachukai,AZ,Justin Fowler2015-03-18 00:00:0024.0MaleOtherL...,0.807018,Justin Fowler2015-03-20 00:00:0024.0MaleOtherR...
2172,171,Joshua Beal,2016-11-05,25.0,Male,Black,Chicago,IL,Joshua Beal2016-11-05 00:00:0025.0MaleBlackChi...,0.807339,Cleotha Mitchell2016-11-25 00:00:0037.0MaleBla...
1250,1092,"Cory Lee ""Jasper"" Henderson",2016-01-17,31.0,Male,White,Holladay,UT,"Cory Lee ""Jasper"" Henderson2016-01-17 00:00:00...",0.808824,Corey Lee Henderson2016-01-17 00:00:0031.0Male...


In [32]:
# what does WAPO report that FBI missed?
# also slow
wapo[['score','fbi_id']] = wapo['id'].apply(lambda x: best_match(x, fbi['id'])).apply(pd.Series)

In [41]:
wapo.sort('score')[30:35]

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,index,name,date,age,gender,race,city,state,id,score,fbi_id
1646,1646,TK TK,2016-09-01,NaN,Male,Hispanic,Huntington Park,CA,TK TK2016-09-01 00:00:00nanMaleHispanicHunting...,0.753846,Name withheld by police2016-09-01 00:00:000.0M...
711,711,Scott Beech,2015-09-19,57.0,Male,White,Washington County,AL,Scott Beech2015-09-19 00:00:0057.0MaleWhiteWas...,0.756303,Scott Beech2015-09-18 00:00:0057.0MaleWhiteSt....
924,924,Sammy Monroe Echols,2015-12-09,51.0,Male,Other,Pryor,OK,Sammy Monroe Echols2015-12-09 00:00:0051.0Male...,0.756303,Sammy Echols2015-12-09 00:00:0051.0MaleWhiteSp...
688,687,Bruce Santino,2015-09-10,35.0,Male,Other,Fontana,CA,Bruce Santino2015-09-10 00:00:0035.0MaleOtherF...,0.756757,Santino Burce2015-09-10 00:00:0034.0MaleHispan...
176,177,Edixon Franco,2015-03-10,37.0,Male,Hispanic,Ontario,CA,Edixon Franco2015-03-10 00:00:0037.0MaleHispa...,0.758065,Luis Guillen Wenceslao2015-09-05 00:00:0032.0M...


In [ ]:
# some other method....